https://github.com/robertpenner/nepali_verb/tree/master

In [1]:

class NepaliVerb:
    """
    यो class ले नेपाली क्रियापद (verb) र त्यसको मूल धातु (root) लाई ह्यान्डल गर्छ।
    """
    DEFAULT_INFINITIVE = 'गर्नु'

    def __init__(self, infinitive=None, past_root=None):
        if not infinitive:
            self.infinitive = self.DEFAULT_INFINITIVE
        else:
            self.infinitive = infinitive
        self._past_root = past_root

    @property
    def root(self):
        """क्रियापदको 'नु' हटाएर मूल धातु निकाल्छ।"""
        return self.infinitive[:-2]

    @property
    def past_root(self):
        """क्रियापदको भूतकालको धातु निकाल्छ।"""
        if self._past_root:
            return self._past_root
        
        # 'पकाउनु' जस्ता क्रियापदका लागि
        if self.root.endswith('ाउ'):
            return self.root[:-1] # अन्तिम 'उ' हटाउने
        
        return self.root

    # 'जानु' (janu) जस्तो अनियमित क्रियाको लागि एउटा class method
    @classmethod
    def JANU(cls):
        return cls('जानु', 'ग')

class Conjugation:
    """
    यो class ले क्रियापदलाई काल (tense), पुरुष (subject) र 
    सकारात्मक/नकारात्मक (positive/negative) आधारमा रूपान्तरण गर्छ।
    """
    # विभिन्न काल र पुरुषका लागि प्रत्यय (suffixes)
    SUFFIXES = {
        'present': {
            'म': ('छु', 'दिन'),
            'तिमी': ('छौ', 'दैनौ'),
            'तपाईं': ('हुन्छ', 'हुन्न'),
            'उ/उनी': ('छ', 'दैन'),
            'उनीहरू': ('छन्', 'दैनन्'),
        },
        'simple_past': {
            'म': ('एँ', 'इनँ'),
            'तिमी': ('यौ', 'एनौ'),
            'तपाईं': ('भयो', 'भएन'),
            'उ/उनी': ('यो', 'एन'),
            'उनीहरू': ('ए', 'एनन्'),
        },
        'completed_past': {
            'म': ('एको छु', 'एको छैन'),
            'तिमी': ('एका छौ', 'एका छैनौ'),
            'तपाईं': ('एको छ', 'एको छैन'),
            'उ/उनी': ('एको छ', 'एको छैन'),
            'उनीहरू': ('एका छन्', 'एका छैनन्'),
        }
    }

    def __init__(self, nepali_verb):
        self.verb = nepali_verb
        self.subject = 'म'
        self.tense = 'present'
        self.negative = False

    def get_conjugated_verb(self):
        """क्रियापदलाई रूपान्तरण गरेर अन्तिम रुप दिन्छ।"""
        
        # भूतकाल (past tense) को लागि कुन धातु प्रयोग गर्ने भन्ने निर्णय
        if self.tense in ['simple_past', 'completed_past']:
            base_root = self.verb.past_root
        else:
            base_root = self.verb.root
            
        # प्रत्यय (suffix) छान्ने
        neg_index = 1 if self.negative else 0
        try:
            suffix = self.SUFFIXES[self.tense][self.subject][neg_index]
        except KeyError:
            return f"नियम भेटिएन (Tense: {self.tense}, Subject: {self.subject})"

        # धातु र प्रत्यय जोड्ने विशेष नियमहरू
        
        # १. खानु -> खान्छु (add half 'न')
        if base_root.endswith('ा') and suffix.startswith('छ'):
            return base_root + 'न्' + suffix

        # २. पकाउनु -> पकाउँछु (add nasalization)
        if base_root.endswith('ाउ') and suffix.startswith('छ'):
            return base_root + 'ँ' + suffix
            
        # ३. खानु (negative) -> खाँदिन
        if base_root.endswith(('ा', 'उ')) and suffix.startswith('द'):
            return base_root + 'ँ' + suffix
            
        # ४. गर्नु (past) -> गरेँ
        if base_root.endswith('र्') and suffix.startswith(('एँ', 'यौ', 'ए')):
             return base_root[:-1] + 'रे' + suffix[1:]


        return base_root + suffix

# --- Jupyter Notebook मा प्रयोगको लागि उदाहरण ---

def run_conjugation_test(description, verb_infinitive, tests):
    print(f"--- {description} ---")
    verb = NepaliVerb(verb_infinitive)
    
    # 'जानु' को विशेष केस
    if verb_infinitive == 'जानु':
        verb = NepaliVerb.JANU()

    for test in tests:
        conj = Conjugation(verb)
        conj.tense = test['tense']
        conj.subject = test['subject']
        conj.negative = test.get('negative', False)
        
        result = conj.get_conjugated_verb()
        status = "✅" if result == test['expected'] else "❌"
        print(f"{status} {verb.infinitive} ({test['subject']}, {test['tense']}): {result} (Expected: {test['expected']})")
    print("-" * 20 + "\n")


# १. साधारण क्रिया: 'गर्नु' (garnu)
garnu_tests = [
    {'subject': 'म', 'tense': 'present', 'expected': 'गर्छु'},
    {'subject': 'म', 'tense': 'present', 'negative': True, 'expected': 'गर्दिन'},
    {'subject': 'तिमी', 'tense': 'present', 'expected': 'गर्छौ'},
    {'subject': 'तिमी', 'tense': 'present', 'negative': True, 'expected': 'गर्दैनौ'},
    {'subject': 'म', 'tense': 'simple_past', 'expected': 'गरेँ'},
    {'subject': 'म', 'tense': 'simple_past', 'negative': True, 'expected': 'गरिनँ'},
    {'subject': 'तिमी', 'tense': 'simple_past', 'expected': 'गर्यौ'},
]
run_conjugation_test("साधारण क्रिया: 'गर्नु'", 'गर्नु', garnu_tests)


# २. अनियमित क्रिया: 'जानु' (janu)
janu_tests = [
    {'subject': 'म', 'tense': 'simple_past', 'expected': 'गएँ'},
    {'subject': 'तिमी', 'tense': 'simple_past', 'expected': 'गयौ'},
    {'subject': 'म', 'tense': 'completed_past', 'expected': 'गएको छु'},
    {'subject': 'तिमी', 'tense': 'completed_past', 'expected': 'एका छौ'}, # Note: 'ग' + 'एका छौ' -> गएका छौ
]
run_conjugation_test("अनियमित क्रिया: 'जानु'", 'जानु', janu_tests)


# ३. एकल-स्वर धातु: 'खानु' (khanu)
khanu_tests = [
    {'subject': 'म', 'tense': 'present', 'expected': 'खान्छु'},
    {'subject': 'म', 'tense': 'present', 'negative': True, 'expected': 'खाँदिन'},
]
run_conjugation_test("एकल-स्वर धातु: 'खानु'", 'खानु', khanu_tests)


# ४. दोहोरो-स्वर धातु: 'पकाउनु' (pakaunu)
pakaunu_tests = [
    {'subject': 'म', 'tense': 'present', 'expected': 'पकाउँछु'},
    {'subject': 'म', 'tense': 'present', 'negative': True, 'expected': 'पकाउँदिन'},
    {'subject': 'म', 'tense': 'simple_past', 'negative': True, 'expected': 'पकाइनँ'},
    {'subject': 'म', 'tense': 'completed_past', 'expected': 'पकाएको छु'},
]
run_conjugation_test("दोहोरो-स्वर धातु: 'पकाउनु'", 'पकाउनु', pakaunu_tests)

--- साधारण क्रिया: 'गर्नु' ---
✅ गर्नु (म, present): गर्छु (Expected: गर्छु)
✅ गर्नु (म, present): गर्दिन (Expected: गर्दिन)
✅ गर्नु (तिमी, present): गर्छौ (Expected: गर्छौ)
✅ गर्नु (तिमी, present): गर्दैनौ (Expected: गर्दैनौ)
❌ गर्नु (म, simple_past): गररेँ (Expected: गरेँ)
❌ गर्नु (म, simple_past): गर्इनँ (Expected: गरिनँ)
❌ गर्नु (तिमी, simple_past): गररेौ (Expected: गर्यौ)
--------------------

--- अनियमित क्रिया: 'जानु' ---
✅ जानु (म, simple_past): गएँ (Expected: गएँ)
✅ जानु (तिमी, simple_past): गयौ (Expected: गयौ)
✅ जानु (म, completed_past): गएको छु (Expected: गएको छु)
❌ जानु (तिमी, completed_past): गएका छौ (Expected: एका छौ)
--------------------

--- एकल-स्वर धातु: 'खानु' ---
✅ खानु (म, present): खान्छु (Expected: खान्छु)
✅ खानु (म, present): खाँदिन (Expected: खाँदिन)
--------------------

--- दोहोरो-स्वर धातु: 'पकाउनु' ---
✅ पकाउनु (म, present): पकाउँछु (Expected: पकाउँछु)
✅ पकाउनु (म, present): पकाउँदिन (Expected: पकाउँदिन)
✅ पकाउनु (म, simple_past): पकाइनँ (Expected: पकाइनँ)
